### KEGG

<Response [200]>

### BioCyc

In [8]:
from modelseedpy_ext.biocyc.api import BiocycAPICached

"""
06/20/2023
BiocycQuery.Pathways 4205
BiocycQuery.Proteins 24391
BiocycQuery.Reactions 20552
BiocycQuery.Compounds 24705
BiocycQuery.Genes 15514
BiocycQuery.Organisms 9165
"""
    
api = BiocycAPICached(cache='.', 
                      #email='fliu@deb.uminho.pt', password='B&5saj.fPx'
                     )
for q in [
    #BiocycQuery.Pathways, BiocycQuery.Proteins, BiocycQuery.Reactions, BiocycQuery.Compounds,
    #BiocycQuery.Genes, 
    #BiocycQuery.Organisms,
    
]:
    res = api.query(q)
    print(q, len(res))
#api.fetch('PWY-7822')
#res = api.fetch('RXN-15479')
api.fetch('CHITOBIOSE')

b'<?xml version="1.0" encoding="iso-8859-1"?>\n<ptools-xml ptools-version=\'27.0\' xml:base=\'http://BioCyc.org/getxml?META:CHITOBIOSE\'><metadata><url>http://BioCyc.org/</url>\n<service_name>getxml</service_name>\n<query>META:CHITOBIOSE</query>\n<num_results>1</num_results>\n<PGDB orgid=\'META\' version=\'27.0\'><species datatype=\'string\'>MetaCyc</species>\n</PGDB>\n</metadata>\n<Compound ID=\'META:CHITOBIOSE\' orgid=\'META\' frameid=\'CHITOBIOSE\' detail=\'full\' class=\'true\'><parent><Compound resource=\'getxml?META:Chitodextrins\' orgid=\'META\' frameid=\'Chitodextrins\' class=\'true\'/></parent>\n<parent><Compound resource=\'getxml?META:Disaccharides\' orgid=\'META\' frameid=\'Disaccharides\' class=\'true\'/></parent>\n<cml><molecule id=\'CHITOBIOSE\' title=\'N,N%27-diacetylchitobiose\' formalCharge=\'0\'><atomArray><atom id=\'CHITOBIOSE-atom1\' elementType=\'C\' x2=\'3.572e+7\' y2=\'1.2375e+7\'/><atom id=\'CHITOBIOSE-atom2\' elementType=\'C\' x2=\'7145000.0\' y2=\'2.8872002e+7

In [60]:
from modelseedpy_ext.biocyc.parser import BiocycParserPass, XmlParseMethod, BiocycParserSingle, BiocycParserDblink
import lxml.etree as et

class BiocycParseBase:

    def __init__(self, end_tag):
        self.end_tag = end_tag
        self.tag_capture = {}

    def _parse(self, elem, parser):
        body = dict(elem.attrib)
        res = {}
        for action, elem in parser:
            t = (action, elem.tag)
            # print('BiocycParseEnzymaticReaction', t)
            if t in self.tag_capture:
                p, method = self.tag_capture[t]
                d = p.parse(elem, parser)
                attr = None
                if attr == None:
                    attr = t[1]
                if method == XmlParseMethod.UNIQUE:
                    if attr in res:
                        raise Exception(f'non unique capture {attr}')
                    else:
                        res[attr] = d
                elif method == XmlParseMethod.UNIQUE_E:
                    if attr in res:
                        raise Exception(f'non unique capture {attr}')
                    else:
                        res[attr] = (dict(elem.attrib), elem.text)
                elif method == XmlParseMethod.LIST:
                    if attr not in res:
                        res[attr] = []
                    res[attr].append(d)
                else:
                    raise Exception(f'method not implemented {method}')
            elif action == 'end' and elem.tag == self.end_tag:
                return body, res
            else:
                raise Exception(f'bad traversal expected: {self.tag_capture.keys()}, found {t}')
                
class BiocycParseTag:
    
    def __init__(self, tag):
        self.tag = tag
    
    def parse(self, elem, parser):
        attr = dict(elem.attrib)
        text = elem.text
        for action, elem in parser:
            if action == 'end' and elem.tag == self.tag:
                return attr, text
            else:
                raise Exception('!!!')
                
class BiocycXmlParser(BiocycParseBase):
    
    def __init__(self, tag):
        super().__init__(tag)
        self.tag = tag
    
    def parse(self, h):
        parser = et.iterparse(h, events=("end", "start"))
        for action, elem in parser:
            tag = elem.tag
            if action == "start" and elem.tag == self.tag:
                return super()._parse(elem, parser)
            
class BiocycCompoundParser(BiocycXmlParser):
    
    def __init__(self):
        super().__init__('Compound')
        self.tag_capture = {
            ('start', 'parent'): (BiocycParserSingle('Compound', 'parent'), XmlParseMethod.LIST),
            ('start', 'cml'): (BiocycParserPass('cml'), XmlParseMethod.UNIQUE),
            ('start', 'appears-in-right-side-of'): (BiocycParserPass('appears-in-right-side-of'), XmlParseMethod.UNIQUE),
            ('start', 'inchi'): (BiocycParseTag('inchi'), XmlParseMethod.UNIQUE),
            ('start', 'synonym'): (BiocycParseTag('synonym'), XmlParseMethod.LIST),
            ('start', 'molecular-weight'): (BiocycParseTag('molecular-weight'), XmlParseMethod.UNIQUE),
            ('start', 'monoisotopic-mw'): (BiocycParseTag('monoisotopic-mw'), XmlParseMethod.UNIQUE),
            ('start', 'dblink'): (BiocycParserDblink(), XmlParseMethod.LIST),
            ('start', 'inchi-key'): (BiocycParserPass('inchi-key'), XmlParseMethod.UNIQUE),
            ('start', 'common-name'): (BiocycParseTag('common-name'), XmlParseMethod.UNIQUE),
            ('start', 'gibbs-0'): (BiocycParserPass('gibbs-0'), XmlParseMethod.UNIQUE),
            ('start', 'credits'): (BiocycParserPass('credits'), XmlParseMethod.UNIQUE),
            ('start', 'comment'): (BiocycParserPass('comment'), XmlParseMethod.UNIQUE),
            ('start', 'appears-in-left-side-of'): (BiocycParserPass('appears-in-left-side-of'), XmlParseMethod.UNIQUE),
        }
        
from io import BytesIO
o = BiocycCompoundParser().parse(BytesIO(api.fetch('CHITOBIOSE')))
o

({'ID': 'META:CHITOBIOSE',
  'orgid': 'META',
  'frameid': 'CHITOBIOSE',
  'detail': 'full',
  'class': 'true'},
 {'parent': [({'resource': 'getxml?META:Chitodextrins',
     'orgid': 'META',
     'frameid': 'Chitodextrins',
     'class': 'true'},
    None),
   ({'resource': 'getxml?META:Disaccharides',
     'orgid': 'META',
     'frameid': 'Disaccharides',
     'class': 'true'},
    None)],
  'cml': None,
  'appears-in-right-side-of': None,
  'inchi': ({'datatype': 'string'},
   'InChI=1S/C16H28N2O11/c1-5(21)17-9-13(25)14(8(4-20)27-15(9)26)29-16-10(18-6(2)22)12(24)11(23)7(3-19)28-16/h7-16,19-20,23-26H,3-4H2,1-2H3,(H,17,21)(H,18,22)/t7-,8-,9-,10-,11-,12-,13-,14-,15?,16+/m1/s1'),
  'synonym': [({'datatype': 'string'}, 'diacetylchitobiose'),
   ({'datatype': 'string'},
    '<i>N</i>-[2-[5-acetylamino-4,6-dihydroxy-2-(hydroxymethyl)tetrahydropyran-3-yl]oxy-4,5-dihyrdoxy-6-(hydroxymethyl)tetrahydropyran-3-yl]acetamide'),
   ({'datatype': 'string'}, 'CBS'),
   ({'datatype': 'string'}, 'chito

In [113]:
res = api.query(BiocycQuery.Pathways)

In [ ]:
s = requests.Session() # create session
    # Post login credentials to session:
    s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'[email]', 'password':'[password]'})
    # Issue web service request:
    r = s.get('https://websvc.biocyc.org/[request]')

In [119]:
res = api.fetch('RXN-15479')

TypeError: cache_fetch() missing 1 required positional argument: 'object_type'

In [61]:
def _detect_class(resp):
    
    parser = ET.iterparse(BytesIO(resp.content), events=("end", "start"))
    for action, elem in parser:
        if action == "end" and elem.tag == "metadata":
            for action, elem in parser:
                if action == 'start':
                    return elem.tag

    raise Exception("unable to detect class")
_detect_class(res)

'Reaction'

In [136]:
#print(res.content.decode('utf-8'))

In [49]:
from enum import Enum
class BiocycQuery(Enum):
    Pathways = 'Pathway'
    Reactions = 'Reaction'
    Compounds = 'Compound'
    Proteins = 'Protein'
    Genes = 'Gene'
    Organisms = 'Organism'
BiocycQuery.Pathways.value

'Pathway'

In [25]:
from io import BytesIO
import lxml.etree as ET

def _to_list(resp):
    res = {}
    
    parser = ET.iterparse(BytesIO(resp.content), events=("end", "start"))
    for action, elem in parser:
        if action == "end" and elem.tag == "metadata":
            for action, elem in parser:
                if action == 'start':
                    if elem.tag not in res:
                        res[elem.tag] = []
                    res[elem.tag].append([dict(elem.attrib), elem.text])
                elif action == "end" and elem.tag == "ptools-xml":
                    return res
    
    raise Exception("unexpected ending")

def _detect_class(xml_str):
    clazz = None
    return clazz

l = _to_list(res)
len(l['Pathway'])

4205

### NCBI Eutils

In [ ]:
https://websvc.biocyc.org/xmlquery?query=[QUERY]&detail=[none|low|full]